In [2]:
import numpy as np
import openmm as mm
import mdtraj as md
from openmm import unit
import openmm.app as app
import os
from openmmtools import states, mcmc
import openmmtools
import os
import os.path as osp
from ReplicaExchangeProtocol import ReplicaExchange


if osp.exists('output-1.nc'): os.system('rm output-1.nc')

In [3]:
forcefield=['amber99sbildn.xml','tip3p.xml']
#forcefield=['amber96.xml','tip3p.xml']
constraints = app.HBonds
nonbonded_cutoff = 0.9*unit.nanometer
switch_distance = 0.75*unit.nanometer
nonbonded_method = app.PME
implicit_solvent = False
solvated = True
hydrogen_mass = None

# Define initial constants of the input pdb file
friction = 0.1 / unit.picoseconds
pressure = 1.0 * unit.atmosphere
temperature = 300*unit.kelvin
timestep = 2.0 * unit.femtosecond

# Load pdb file
pdb = app.PDBFile('chi_sys.pdb')
traj = md.load('chi_sys.pdb')
modeller = app.modeller.Modeller(pdb.topology, pdb.positions)



# Define system
create_system_kwargs = dict(
    removeCMMotion=True,
    nonbondedMethod=nonbonded_method,
    nonbondedCutoff=nonbonded_cutoff,
    switchDistance=switch_distance,
    constraints=constraints,
    hydrogenMass=hydrogen_mass,
    rigidWater=True,
)


ff = app.ForceField(*forcefield)
system = ff.createSystem(modeller.topology, **create_system_kwargs)


#sampler_state = mcmc.SamplerState(positions=system.positions)
#thermodynamic_state = mcmc.ThermodynamicState(system=system, temperature=298*unit.kelvin)


barostat = mm.MonteCarloBarostat(pressure, temperature)
force_id = system.addForce(barostat)

In [4]:
platform = mm.Platform.getPlatformByName("CUDA")
platform_properties = {"DeviceIndex": "0", "Precision": "mixed"}

n_replicas = 4

protocol = {'temperature': [300,302,304,306,315] * unit.kelvin}
thermodynamic_states = states.create_thermodynamic_state_protocol(system,protocol)



sampler_states = list()
for i_t,_ in enumerate(thermodynamic_states):
    sampler_states.append(openmmtools.states.SamplerState(positions=modeller.positions))

langevin_move = mcmc.LangevinSplittingDynamicsMove(
    timestep = timestep,
    collision_rate = friction,
    n_steps = 1000,
    reassign_velocities=False,
    n_restart_attempts=20,
    splitting="V R O R V"
)

# Define class for replica exchange
parallel_tempering = ReplicaExchange(
    thermodynamic_states=thermodynamic_states,
    sampler_states=sampler_states,
    mcmc_move=langevin_move
)

In [5]:
positions, forces, acceptance = parallel_tempering.run(5000,save=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [6]:
np.diag(acceptance,1) / np.diag(acceptance,-1)

array([0.        , 0.        , 0.1610338 , 0.19917012])

In [7]:
acceptance

array([[  0.,   0., 111., 126.,  98.],
       [472.,   0.,   0.,   1.,   1.],
       [517., 512.,   0.,  81., 112.],
       [531., 469., 503.,   0.,  96.],
       [518., 486., 510., 482.,   0.]])

In [9]:
platform = mm.Platform.getPlatformByName("CUDA")
platform_properties = {"DeviceIndex": "0", "Precision": "mixed"}

n_replicas = 4

protocol = {'temperature': [300,302,304,306,308] * unit.kelvin}
thermodynamic_states = states.create_thermodynamic_state_protocol(system,protocol)



sampler_states = list()
for i_t,_ in enumerate(thermodynamic_states):
    sampler_states.append(openmmtools.states.SamplerState(positions=modeller.positions))

langevin_move = mcmc.LangevinSplittingDynamicsMove(
    timestep = timestep,
    collision_rate = friction,
    n_steps = 1000,
    reassign_velocities=False,
    n_restart_attempts=20,
    splitting="V R O R V"
)

# Define class for replica exchange
parallel_tempering = ReplicaExchange(
    thermodynamic_states=thermodynamic_states,
    sampler_states=sampler_states,
    mcmc_move=langevin_move
)

In [10]:
positions, forces, acceptance = parallel_tempering.run(5000,save=True)

print(np.diag(acceptance,1) / np.diag(acceptance,-1))
print(acceptance)

0
100


KeyboardInterrupt: 

In [11]:
positions.shape

(5, 5000, 20963, 3)

In [13]:
99 % 10

9

In [2]:
import numpy as np
n_replicas = 6
min_T = 300
max_T = 320
np.geomspace(min_T, max_T, n_replicas)

array([300.        , 303.89741047, 307.84545363, 311.84478727,
       315.89607773, 320.        ])

In [6]:
for i in range(100):
    if (1+i) % 10 == 0:
        print(f'{i+1} of {100}, value of i = {i}')


10 of 100, value of i = 9
20 of 100, value of i = 19
30 of 100, value of i = 29
40 of 100, value of i = 39
50 of 100, value of i = 49
60 of 100, value of i = 59
70 of 100, value of i = 69
80 of 100, value of i = 79
90 of 100, value of i = 89
100 of 100, value of i = 99


In [8]:
import numpy as np 

acceptance = np.load('acceptance1.npy')
print(np.diag(acceptance,1) / np.diag(acceptance,-1))
print(acceptance)

[0.00920245 0.05521472 0.02167183 0.02312139 0.03809524]
[[  0.   3.   3.   5.  12.   7.]
 [326.   0.  18.  10.   7.  15.]
 [350. 326.   0.   7.   3.   2.]
 [337. 317. 323.   0.   8.   7.]
 [337. 369. 357. 346.   0.  12.]
 [338. 340. 341. 278. 315.   0.]]


In [21]:
a = 
i = np.sort(np.random.choice(range(10),1,replace=False))
j = i + 1
k = i - 1

neighs = a[abs(a - r) == 1]

if j is not in range(10):
    print(f'i: {i}, j: {k}')

SyntaxError: invalid syntax (1100248564.py, line 5)